# Comparing Before Budgeted CPU versus After Budgeted CPU

### This creates the raw "Red and Green" sheet master data

### Overall Process

- Get current or "After" budgeted CPU file
- Get "Before" budgeted CPU file
- Merge between the 2 using full OUTER JOIN
- Create 2 new columns:
    1. After Budgeted CPU minus Before Budgeted CPU (at group-subgroup level)
    2. After Budgeted CPU minus Before Budgeted CPU (at GraphCat level)
- Export to Excel and save in ```\\localhost\aqgbudget2\Cost\Reserve Adjustments\Reports\Normal Reserve Balance Verification\RVMS_Before_After_Checks``` folder

In [1]:
from datetime import datetime
from dateutil import relativedelta
from pathlib import Path
import os
import pandas as pd
import pyodbc
import scrapbook as sb
import time
from win10toast import ToastNotifier
pd.options.display.max_rows=1000
pd.options.display.max_columns=100

### Enter CLA Claim Month:

In [ ]:
# If using papermill, have to comment this out.  It doesn't support getting input from the user
# CLA_MONTH = input("Enter CLA Claim Month ('YYYYMM'): ")

In [2]:
CLA_MONTH = '201901'

In [3]:
start_time = time.time()

### To obtain previous month's CPUs, need to create variable for obtaining the current CLA month minus one month:

In [4]:
current_date = datetime(int(CLA_MONTH[:4]),int(CLA_MONTH[-2:]),1,0,0,0,0)

In [5]:
current_date

datetime.datetime(2019, 1, 1, 0, 0)

In [6]:
current_date_minus_one_month = (current_date + relativedelta.relativedelta(months=-1)).strftime('%Y%m')

In [7]:
current_date_minus_one_month

'201812'

### Define where to save the Red and Green raw data file based on CLA claim month:

In [8]:
base_dir = "//localhost/aqgbudget2/Cost/Reserve Adjustments/Reports/Normal Reserve Balance Verification/RVMS_Before_After_Checks"
p = Path(base_dir)
save_dir = p / CLA_MONTH
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

### Define where to retrieve the current budgeted CPUs ("after" CPUs):

In [9]:
current_cpu_dir = Path("//localhost/aqgbudget2/Cost/Reserve Adjustments/Reports/Normal Reserve Balance Verification/RVMS_Current_Budgeted_CPUs/" 
                       + CLA_MONTH + "/All_Plants_Budgeted_CPU_By_Group_SubGroup.xlsx")

#### Now fetch the "after" CPUs:

In [10]:
cpu_after = pd.read_excel(current_cpu_dir)

In [11]:
cpu_after.shape

(58680, 14)

In [15]:
cpu_after['Group-SubGroup'] = cpu_after['GRP_NM'].map(str) + ' - ' + cpu_after['SUBGRP_NM'].map(str)

In [16]:
cpu_after.head()

,GraphCatID,GraphCatDesc,GRP_NM,SUBGRP_NM,Budgeted_CPU_SubGroup_Level,Planned_Sales_RVMS,Budgeted_CPU_GC_Level,Orig_Saturation_CPU_GC_Level,Cum_Actual_CPU_GC_Level,RVMS_Claim_Month,ModelYear,Factory,ModelName,DestCode,Group-SubGroup
0,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.834396,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Calipers/Disk/Drums/Pads
1,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.891899,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Drive Shaft
2,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.048017,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Hand/cable Brakes
3,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.049997,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Master Cylinder
4,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.325561,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Steering Vibration


In [17]:
after_column_names = [col + '_After' for col in cpu_after.columns]

In [18]:
after_column_names

['GraphCatID_After',
 'GraphCatDesc_After',
 'GRP_NM_After',
 'SUBGRP_NM_After',
 'Budgeted_CPU_SubGroup_Level_After',
 'Planned_Sales_RVMS_After',
 'Budgeted_CPU_GC_Level_After',
 'Orig_Saturation_CPU_GC_Level_After',
 'Cum_Actual_CPU_GC_Level_After',
 'RVMS_Claim_Month_After',
 'ModelYear_After',
 'Factory_After',
 'ModelName_After',
 'DestCode_After',
 'Group-SubGroup_After']

In [19]:
cpu_after.columns = after_column_names

In [20]:
cpu_after.head()

,GraphCatID_After,GraphCatDesc_After,GRP_NM_After,SUBGRP_NM_After,Budgeted_CPU_SubGroup_Level_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,RVMS_Claim_Month_After,ModelYear_After,Factory_After,ModelName_After,DestCode_After,Group-SubGroup_After
0,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.834396,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Calipers/Disk/Drums/Pads
1,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.891899,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Drive Shaft
2,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.048017,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Hand/cable Brakes
3,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.049997,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Master Cylinder
4,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.325561,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Steering Vibration


#### Get "Before" budgeted CPU file and rename columns:

In [21]:
conn_str = (
            r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
            r'DBQ=\\localhost\aqgbudget2\Cost\Reserve Adjustments\Reports\databases\RVMS.accdb;'
           )
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()

sql = """
SELECT
    *
    
FROM tbl_Historical_Budgeted_CPU

WHERE
    RVMS_Claim_Month = ?
"""

try:
    cpu_before = pd.read_sql(sql, cnxn, params=[current_date_minus_one_month])
    #cpu_before = pd.read_sql(sql, cnxn)
    
    # Close connections
    cursor.close()
    cnxn.close()
except:
    print("Error connecting to database")
    cursor.close()
    cnxn.close()

In [22]:
cpu_before.shape

(51720, 15)

In [23]:
cpu_before.head()

,ID,GraphCatID,GraphCatDesc,GRP_NM,SUBGRP_NM,Budgeted_CPU_SubGroup_Level,Planned_Sales_RVMS,Budgeted_CPU_GC_Level,Orig_Saturation_CPU_GC_Level,Cum_Actual_CPU_GC_Level,RVMS_Claim_Month,ModelYear,Factory,ModelName,DestCode
0,170914,965,R HMA ACCORD 2010 V6 KA,Denso,Washer System,0.000000,18539,241.08,None,None,201812,2010,HMA,ACCORD,KA
1,170915,965,R HMA ACCORD 2010 V6 KA,Denso,Wiper Motor,0.029453,18539,241.08,None,None,201812,2010,HMA,ACCORD,KA
2,170916,965,R HMA ACCORD 2010 V6 KA,OBD,Fuel Pump,0.109686,18539,241.08,None,None,201812,2010,HMA,ACCORD,KA
3,170917,965,R HMA ACCORD 2010 V6 KA,Chassis,Wheels,1.572711,18539,241.08,None,None,201812,2010,HMA,ACCORD,KA
4,170918,965,R HMA ACCORD 2010 V6 KA,OBD,Missfire,8.216182,18539,241.08,None,None,201812,2010,HMA,ACCORD,KA


In [24]:
before_column_names = [col + '_Before' for col in cpu_before.columns]

In [25]:
cpu_before.columns = before_column_names

In [26]:
cpu_before.head()

,ID_Before,GraphCatID_Before,GraphCatDesc_Before,GRP_NM_Before,SUBGRP_NM_Before,Budgeted_CPU_SubGroup_Level_Before,Planned_Sales_RVMS_Before,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_Before,Cum_Actual_CPU_GC_Level_Before,RVMS_Claim_Month_Before,ModelYear_Before,Factory_Before,ModelName_Before,DestCode_Before
0,170914,965,R HMA ACCORD 2010 V6 KA,Denso,Washer System,0.000000,18539,241.08,None,None,201812,2010,HMA,ACCORD,KA
1,170915,965,R HMA ACCORD 2010 V6 KA,Denso,Wiper Motor,0.029453,18539,241.08,None,None,201812,2010,HMA,ACCORD,KA
2,170916,965,R HMA ACCORD 2010 V6 KA,OBD,Fuel Pump,0.109686,18539,241.08,None,None,201812,2010,HMA,ACCORD,KA
3,170917,965,R HMA ACCORD 2010 V6 KA,Chassis,Wheels,1.572711,18539,241.08,None,None,201812,2010,HMA,ACCORD,KA
4,170918,965,R HMA ACCORD 2010 V6 KA,OBD,Missfire,8.216182,18539,241.08,None,None,201812,2010,HMA,ACCORD,KA


In [27]:
try:
    assert cpu_before.shape[0] == cpu_after.shape[0]
except:
    toaster = ToastNotifier()
    toaster.show_toast("### ERROR ###",
                   "Number of rows don't match between CPU after and CPU before data sets",
                   icon_path=None,
                   duration=5)
    print('ERROR!!! - Number of rows do not match between CPU after and CPU before data sets')

ERROR!!! - Number of rows do not match between CPU after and CPU before data sets


### Merge the after CPU data set with the before CPU data set

In [28]:
cpu_before_after_merge = pd.merge(cpu_after, cpu_before, how='outer', 
                                  left_on=['GraphCatID_After','GRP_NM_After','SUBGRP_NM_After'], 
                                  right_on=['GraphCatID_Before','GRP_NM_Before','SUBGRP_NM_Before']
                                 )

In [29]:
cpu_before_after_merge.shape

(58680, 30)

In [30]:
cpu_before_after_merge.head()

,GraphCatID_After,GraphCatDesc_After,GRP_NM_After,SUBGRP_NM_After,Budgeted_CPU_SubGroup_Level_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,RVMS_Claim_Month_After,ModelYear_After,Factory_After,ModelName_After,DestCode_After,Group-SubGroup_After,ID_Before,GraphCatID_Before,GraphCatDesc_Before,GRP_NM_Before,SUBGRP_NM_Before,Budgeted_CPU_SubGroup_Level_Before,Planned_Sales_RVMS_Before,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_Before,Cum_Actual_CPU_GC_Level_Before,RVMS_Claim_Month_Before,ModelYear_Before,Factory_Before,ModelName_Before,DestCode_Before
0,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.834396,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Calipers/Disk/Drums/Pads,180847.0,449.0,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.834394,94558.0,101.97,None,None,201812,2010.0,HMI,CIVIC,KA
1,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.891899,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Drive Shaft,180793.0,449.0,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.891603,94558.0,101.97,None,None,201812,2010.0,HMI,CIVIC,KA
2,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.048017,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Hand/cable Brakes,180832.0,449.0,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.048016,94558.0,101.97,None,None,201812,2010.0,HMI,CIVIC,KA
3,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.049997,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Master Cylinder,180790.0,449.0,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.049997,94558.0,101.97,None,None,201812,2010.0,HMI,CIVIC,KA
4,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.325561,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Steering Vibration,180853.0,449.0,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.325561,94558.0,101.97,None,None,201812,2010.0,HMI,CIVIC,KA


### Create columns that represent the before and after CPUs at GraphCat level, subgroup level, and total adjustment costs

In [31]:
cpu_before_after_merge['Budgeted_CPU_SubGroup_Level_After_Minus_Before'] = cpu_before_after_merge['Budgeted_CPU_SubGroup_Level_After'] \
                                                            - cpu_before_after_merge['Budgeted_CPU_SubGroup_Level_Before']
cpu_before_after_merge['Budgeted_CPU_GC_Level_After_Minus_Before'] = cpu_before_after_merge['Budgeted_CPU_GC_Level_After'] \
                                                                     - cpu_before_after_merge['Budgeted_CPU_GC_Level_Before']
cpu_before_after_merge['CPU_DIFF_SubGroup_Level_x_SALES'] = cpu_before_after_merge['Budgeted_CPU_SubGroup_Level_After_Minus_Before'] \
                                                      * cpu_before_after_merge['Planned_Sales_RVMS_After']
cpu_before_after_merge['CPU_DIFF_GC_LEVEL_x_SALES'] = cpu_before_after_merge['Budgeted_CPU_GC_Level_After_Minus_Before'] \
                                                      * cpu_before_after_merge['Planned_Sales_RVMS_After']

In [32]:
cpu_before_after_merge.head()

,GraphCatID_After,GraphCatDesc_After,GRP_NM_After,SUBGRP_NM_After,Budgeted_CPU_SubGroup_Level_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,RVMS_Claim_Month_After,ModelYear_After,Factory_After,ModelName_After,DestCode_After,Group-SubGroup_After,ID_Before,GraphCatID_Before,GraphCatDesc_Before,GRP_NM_Before,SUBGRP_NM_Before,Budgeted_CPU_SubGroup_Level_Before,Planned_Sales_RVMS_Before,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_Before,Cum_Actual_CPU_GC_Level_Before,RVMS_Claim_Month_Before,ModelYear_Before,Factory_Before,ModelName_Before,DestCode_Before,Budgeted_CPU_SubGroup_Level_After_Minus_Before,Budgeted_CPU_GC_Level_After_Minus_Before,CPU_DIFF_SubGroup_Level_x_SALES,CPU_DIFF_GC_LEVEL_x_SALES
0,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.834396,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Calipers/Disk/Drums/Pads,180847.0,449.0,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.834394,94558.0,101.97,None,None,201812,2010.0,HMI,CIVIC,KA,2.020800e-06,0.04,0.191083,3782.32
1,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.891899,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Drive Shaft,180793.0,449.0,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.891603,94558.0,101.97,None,None,201812,2010.0,HMI,CIVIC,KA,2.962053e-04,0.04,28.008581,3782.32
2,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.048017,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Hand/cable Brakes,180832.0,449.0,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.048016,94558.0,101.97,None,None,201812,2010.0,HMI,CIVIC,KA,2.518000e-07,0.04,0.023810,3782.32
3,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.049997,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Master Cylinder,180790.0,449.0,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.049997,94558.0,101.97,None,None,201812,2010.0,HMI,CIVIC,KA,4.350000e-07,0.04,0.041133,3782.32
4,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.325561,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Steering Vibration,180853.0,449.0,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.325561,94558.0,101.97,None,None,201812,2010.0,HMI,CIVIC,KA,1.329000e-07,0.04,0.012567,3782.32


### Define file name format:

In [33]:
date_hour_stamp = time.strftime('%Y-%m-%d_%H_%M')
file_name = 'All_Plants_Before_After_Budgeted_CPUs_' + date_hour_stamp + '.xlsx'

### Write/save file to designated network share drive location:

In [34]:
cpu_before_after_merge.to_excel(save_dir / file_name, index=False)

### Now, we need to "glue" the location of the saved file location to this notebook so that another notebook can retrieve/reference from it:

In [35]:
str(save_dir / file_name)

'\\\\localhost\\aqgbudget2\\Cost\\Reserve Adjustments\\Reports\\Normal Reserve Balance Verification\\RVMS_Before_After_Checks\\201901\\All_Plants_Before_After_Budgeted_CPUs_2019-03-11_14_20.xlsx'

In [36]:
sb.glue("path_to_red_green_sheet_excel_file", str(save_dir / file_name))

### Send Windows Toast notification when script completes

In [37]:
toaster = ToastNotifier()
toaster.show_toast("### Before vs After CPU Status ###",
                   "Successfuly compared before CPUs with after CPU adjustments",
                   icon_path="images/honda_logo.ico",
                   duration=5)

ERROR:root:Some trouble with the icon (D:\jupyter\rvms\dev\images\honda_logo.ico): (2, 'LoadImage', 'The system cannot find the file specified.')


True